# Gan Baseline

#### NOTE: Need to activate genomelake environment before this code. Simply type 'genomelake' in terminal.

In [19]:
%env CUDA_VISIBLE_DEVICES=4,5
import os, sys
sys.path.append("..")
import random
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats.stats import pearsonr,spearmanr
import tensorflow as tf

env: CUDA_VISIBLE_DEVICES=4,5


In [20]:
window_size = 5001
sample_num = 10000

In [21]:
# retrieve data
data = Data_Directories()
print data.intervals.keys()
print data.input_atac['day0'].keys()
print data.output_histone['day0'].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


In [22]:
# get intervals for day0 data
day0_intervals = list(BedTool(data.intervals['day0']))
print '# of Intervals Extracted for day0: {}'.format(len(day0_intervals))

# of Intervals Extracted for day0: 267226


In [23]:
# create an ArrayExtractor for ATAC-seq for day0 with 140 base pairs
bw_140bp_day0 = ArrayExtractor(data.input_atac['day0']['140'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [24]:
# create a BigWigExtractor for histone makr 'H3K27ac' for day0
bw_histone_mark_day0 = BigwigExtractor(data.output_histone['day0']['H3K27ac'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [25]:
# normalize day0 intervals
normalized_day0_intervals = [normalize_interval(interval, window_size) for interval in day0_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day0 intervals!'

Finished normalizing day0 intervals!


In [26]:
assert (len(day0_intervals)==len(normalized_day0_intervals))
print "Examples of original intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in day0_intervals[:3]]
print "Examples of normalized intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_day0_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 5001
[[123409527, 123414528], [123409441, 123414442], [131906064, 131911065]]


In [27]:
atac_seq_day0 = bw_140bp_day0(normalized_day0_intervals)
print atac_seq_day0.shape

(267226, 5001, 5)


In [28]:
#TODO: put this into utils if possible
def prune_invalid_intervals(intervals, bigwig_file):
    for _interval in intervals[:]:
        try:
            bigwig_file([_interval])
        except:
            intervals.remove(_interval)
            pass
        
print "Before pruning day0: {}".format(len(normalized_day0_intervals))
prune_invalid_intervals(normalized_day0_intervals, bw_140bp_day0)
print "After pruning day0: {}".format(len(normalized_day0_intervals))

Before pruning day0: 267226
After pruning day0: 267226


In [29]:
print "Dimension of ATAC-seq signal: {}".format(bw_140bp_day0(normalized_day0_intervals[:1]).shape)

Dimension of ATAC-seq signal: (1, 5001, 5)


In [30]:
print "Dimension of histone mark signal: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1]).shape)

Dimension of histone mark signal: (1, 5001)


In [31]:
# Replace nan values with zeros and convert it to p-values
histone_mark_day0 = np.nan_to_num(bw_histone_mark_day0(normalized_day0_intervals))
print histone_mark_day0.shape

(267226, 5001)


In [32]:
histone_mark_day0 = np.expand_dims(histone_mark_day0, axis=2)
print histone_mark_day0.shape

(267226, 5001, 1)


In [33]:
print "Example histone mark signal"
print "\tRaw value: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1])[0][:5].reshape(-1))

Example histone mark signal
	Raw value: [ 0.97687  0.97687  0.97687  0.97687  0.97687]


In [34]:
from keras.layers import AveragePooling1D, Input, Dense, Conv1D, Dropout, BatchNormalization, Activation, ZeroPadding1D, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam, SGD

In [35]:
smooth_rate = 0.05
dropout_rate = 0.5
# parameters for first conv layer
hidden_filters_1 = 32
hidden_kernel_size_1 = window_size
# parameters for second conv layer
output_filters = 1
output_kernel_size = 16
# parameters for training
batch_size = 128
num_epochs = 200
evaluation_freq = 10

In [36]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [37]:
model_dir = os.path.join("models", "gan_fixed_smooth_wide_new")
log_dir = os.path.join("logs", "gan_fixed_smooth_wide_new")
srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "gan_fixed_smooth_wide_new")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(srv_dir):
    os.makedirs(srv_dir)

In [65]:
# Our GAN Model (based on vanilla gan)
class GAN():
    def __init__(self, window_size, sample_num):
        # Basic parameters
        # 1) Window size of input/ouput signal
        self.window_size = window_size
        # 2) Number of channels
        self.channels = 5
        # 3) Input and Output shape
        self.input_shape = (self.window_size, self.channels,)
        self.output_shape = (self.window_size, 1,)
        # 4) Total number of samples
        self.sample_num = sample_num
        
        # Adam optimizer for generator
        optimizer = Adam(0.002, beta_1=0.5)
        # Adam optimizer for discriminator
        doptimizer = Adam(0.0005, beta_1=0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
                                   optimizer=doptimizer,
                                   metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy',
                               optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=self.input_shape)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy',
                              optimizer=optimizer)
        print "Combined Model"
        print self.combined.summary()

    def build_generator(self):
        # Generator
        # 1) 32 * window_size Conv1D layers with RELU and Dropout

        noise_shape = self.input_shape
        
        model = Sequential()

        model.add(Conv1D(hidden_filters_1,
                         hidden_kernel_size_1,
                         padding="same",
                         strides=1,
                         input_shape=noise_shape,
                         activation='relu'))
        model.add(Dropout(dropout_rate))

        # 2) 1 * 16 Conv1D layers with Linear
        # NOTE: All same padding
        model.add(Conv1D(output_filters,
                         output_kernel_size,
                         padding='same',
                         strides=1))
        model.add(Activation('linear'))
        
        print "Generator"
        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        # Discriminator
        # 1) 16 * 200 Conv1D with LeakyRelu, Dropout, and BatchNorm
        model = Sequential()
        
        model.add(Conv1D(hidden_filters_1,
                         200,
                         padding="valid",
                         strides=1,
                         input_shape=self.output_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(dropout_rate))
        model.add(BatchNormalization(momentum=0.8))
        
        # 2) Average Pooling, Flatten, Dense, and LeakyRelu
        model.add(AveragePooling1D(25))
        model.add(Flatten())
        model.add(Dense(int(window_size/16)))
        model.add(LeakyReLU(alpha=0.2)) 
        
        # 3) Final output with sigmoid
        model.add(Dense(1, activation='sigmoid'))
        
        print "Discriminator"
        model.summary()

        img = Input(shape=self.output_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, X_train, y_train):
        
        max_pearson = -1.0
        
        # size of the half of the batch
        half_batch = int(batch_size / 2)
            
        d_loss_real, d_loss_fake, g_loss = [1, 0], [1, 0], [1, 0]
            
        for epoch in range(epochs):
            
            # list for storing losses/accuracies for both discriminator and generator
            d_losses, d_accuracies, g_losses = [], [], []
            
            # sufficient number of minibatches for each epoch
            for _minibatch_idx in range(128):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random half batch of images
                dis_idx = np.random.randint(0, y_train.shape[0], half_batch)
                # Get the real images
                imgs = y_train[dis_idx]

                # Generate a half batch of new images
                dis_noise = X_train[dis_idx]
                gen_imgs = self.generator.predict(dis_noise)
                
                # Train the discriminator with label smoothing
                # NOTE: (no smoothing for this version)
                #smoothed_idx = np.random.choice(half_batch, int(half_batch*smooth_rate), replace=False)
                smoothed_labels = np.ones((half_batch, 1))
                #smoothed_labels[smoothed_idx] = 0
                
                # Train the discriminator
                # 1) Concatenate the real images and generated images
                all_imgs = np.concatenate([imgs, gen_imgs])
                # 2) Concatenate the real labels and labels for generated images
                all_labels = np.concatenate([smoothed_labels, np.zeros((half_batch, 1))])
                # 3) Shuffle two lists in same order
                def unison_shuffled_copies(a, b):
                    assert len(a) == len(b)
                    p = np.random.permutation(len(a))
                    return a[p], b[p]
                all_imgs, all_labels = unison_shuffled_copies(all_imgs, all_labels)
                # 4) Train the discriminator for current batch
                d_loss = self.discriminator.train_on_batch(all_imgs, all_labels)
                
                # ---------------------
                #  Train Generator
                # ---------------------
                
                # 1) Randomly choose the batch number of indices
                gen_idx = np.random.randint(0, y_train.shape[0], batch_size)
                # 2) Get the randomly chosen inputs (noises in traditional gan)
                gen_noise = X_train[gen_idx]
                # 3) Create the labels for noises
                # The generator wants the discriminator to label the generated samples as valid (ones)
                valid_y = np.array([1] * batch_size)

                # 4) Train the generator
                # Discriminator is no longer trained at this point
                # Generator do its best to fake discriminator
                g_loss = self.combined.train_on_batch(gen_noise, valid_y)
                
                # ---------------------
                # Store loss and accuracy
                # ---------------------
                d_losses.append(d_loss[0])
                d_accuracies.append(d_loss[1])
                
                g_losses.append(g_loss)
            
            # ---------------------
            # Convert each histories into numpy arrays to get means
            # ---------------------
            d_losses = np.array(d_losses)
            d_accuracies = np.array(d_accuracies)
            g_losses = np.array(g_losses)
            
            # ---------------------
            # Get generator's prediction and compute overall pearson on train set
            # ---------------------
            predictions = self.generator.predict(X_train).flatten()
            avg_pearson = pearsonr(predictions, y_train.flatten())
            print "Pearson R on Train set: {}".format(avg_pearson)
            
            # ---------------------
            # Get generator's prediction and compute overall pearson on validation set
            # ---------------------
            val_predictions = self.generator.predict(X_val).flatten()
            avg_val_pearson = pearsonr(val_predictions, y_val.flatten())
            print "Pearson R on Val set: {}".format(avg_val_pearson)

# ---------------------
#            Old code for taking pearson on each interval and average
#             predictions = self.generator.predict(X_train)
#             pearsons = []
#             for pred_idx in range(len(predictions)):
#                 prediction = predictions[pred_idx]
#                 pearsons.append(pearsonr(prediction, y_train[pred_idx]))
#             avg_pearson = np.array(pearsons).mean()
#             print "Pearson R on Train set: {}".format(avg_pearson)
            
#             val_predictions = self.generator.predict(X_val)
#             val_pearsons = []
#             for val_pred_idx in range(len(val_predictions)):
#                 prediction = val_predictions[val_pred_idx]
#                 val_pearsons.append(pearsonr(prediction, y_val[val_pred_idx]))
#             avg_val_pearson = np.array(val_pearsons).mean()
#             print "Pearson R on Val set: {}".format(avg_val_pearson)  
# ---------------------
            
            # if current pearson on validation set is greatest so far, update the max pearson,
            # and 
            if max_pearson < avg_val_pearson:
                print "Perason on val improved from {} to {}".format(max_pearson, avg_val_pearson)
                _write_1D_deeplift_track(predictions.reshape(int(sample_num*0.8), self.window_size),
                                         normalized_day0_intervals[:int(sample_num*0.8)], os.path.join(srv_dir, 'train'))
                _write_1D_deeplift_track(val_predictions.reshape(int(sample_num*0.2), self.window_size),
                                         normalized_day0_intervals[int(sample_num*0.8):sample_num], os.path.join(srv_dir, 'val'))
                f = open(os.path.join(srv_dir, 'meta.txt'), 'wb')
                f.write(str(epoch) + " " + str(avg_pearson) + "  " + str(avg_val_pearson))
                f.close()
                max_pearson = avg_val_pearson
            
            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_losses.mean(), 100.0*d_accuracies.mean(), g_losses.mean()))

In [66]:
# helper function for computing Pearson R in Keras
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [67]:
print "Fitting the model..."
train_index = int(sample_num*0.8)
val_index = sample_num
test_index = int(sample_num*1.1) 
X_train, y_train = atac_seq_day0[:train_index], histone_mark_day0[:train_index]
X_val, y_val = atac_seq_day0[train_index:val_index], histone_mark_day0[train_index:val_index]
X_test, y_test = atac_seq_day0[val_index:test_index], histone_mark_day0[val_index:test_index]

gan = GAN(window_size, sample_num)
gan.train(num_epochs, batch_size, X_train, y_train)

Fitting the model...
Discriminator
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_28 (Conv1D)           (None, 4802, 32)          6432      
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 4802, 32)          0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 4802, 32)          0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 4802, 32)          128       
_________________________________________________________________
average_pooling1d_10 (Averag (None, 192, 32)           0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 6144)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 312) 

KeyboardInterrupt: 